# Web Scraping
In this project, I performed web scraping on the real estate platform [Zingat](https://www.zingat.com/) to extract house price data for four major cities in Turkey: **Istanbul**, **İzmir**, **Ankara**, and **Antalya**. Using Python and web scraping libraries such as `BeautifulSoup` and `selenium`, I navigated through the website's structure to collect valuable information about house prices and relevant property features. 

This scraped data will serve as the foundation for building a predictive model to estimate house prices. By analyzing the differences in pricing trends across these cities, the project aims to provide insights into regional real estate markets and enhance the accuracy of the predictions.

In [120]:
#pip install selenium

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver

## Istanbul Sale Homes -Web Scraping

In [8]:
links=[]
for page in range(1,31):
    url = "https://www.zingat.com/istanbul-satilik" +'?page='+str(page)
    driver = webdriver.Chrome()  
    driver.get(url) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    soup.prettify()  
    links_html = soup.find_all('a', class_='zl-card-inner')
    for link in links_html:
        new_link ='https://www.zingat.com/'+ link.get('href')
        links.append(new_link)
    driver.quit()

In [9]:
links

['https://www.zingat.com//muslum-emlaktan-avcilar-ambarlida-ici-masrafsiz-2-1-ara-kat-5188304i',
 'https://www.zingat.com//goztepe-soyak-sitesinde-satilik-3-1-ara-kat-daire-5188036i',
 'https://www.zingat.com//bagcilar-gunesli-site-icinde-satilik-2-1-daire-krediye-uygun-5187149i',
 'https://www.zingat.com//avcilar-cihangirde-otoparkli-3-1-kaymakliga-yakin-satilik-5187225i',
 'https://www.zingat.com//satilik-ortakoy-doganbeyli-sitesi-3-1-5186668i',
 'https://www.zingat.com//cavusbasi-y-selim-mh-karanlikdere-cd-satilik-2-800m2-arazi-5186567i',
 'https://www.zingat.com//esenyurt-talatpasa-da-yeni-binada-2-1-arakat-satilik-daire-5185111i',
 'https://www.zingat.com//haramidere-luks-residence-da-deniz-manzarali-4-1-5181524i',
 'https://www.zingat.com//kadikoy-sogutlucesmede-2-1-satilik-daire-5081730i',
 'https://www.zingat.com//b-cekmece-karaagacta-satilik-villa-imarli-arsa-5172292i',
 'https://www.zingat.com//beykoz-yalikoyde-deniz-manzarali-satilik-asansorlu-5-katli-villa-5156452i',
 'http

In [10]:
len(links)

630

In [11]:
expected_features = [
    "Fiyat","İlan Tarihi", "Net m²", "Brüt m²", "Oda Sayısı", "Banyo Sayısı",
    "Binadaki Kat Sayısı", "Bulunduğu Kat", "Bina Yaşı", "Yapının Durumu",
    "Isıtma Tipi", "Kira Getirisi", "Aidat", "Kullanım Durumu", "Krediye Uygun","Tapu Durumu","Görüntülü Ev Gezintisi","Mobilya Durumu",
    "Tapu Durumu","Kimden",'Parsel','Ada',"Konut Şekli"
]
df_istanbul = pd.DataFrame(columns=expected_features)

In [12]:
for link in links:
    driver = webdriver.Chrome()  
    driver.get(link) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    feature_data = {feature: None for feature in expected_features}
    price_tag = soup.find("strong", {"itemprop": "price"})
    feature_data['Fiyat']=price_tag
    for li in soup.find_all("li", class_="col-md-6"):
        key_element = li.find("strong", class_="col-md-6")
        value_element = li.find("span", class_="col-md-6")
        if key_element and value_element:
            key = key_element.get_text(strip=True)
            value = value_element.get_text(strip=True)
            if key in feature_data:  
                feature_data[key] = value       
    new_row = pd.DataFrame([feature_data], columns=expected_features)  
    df_istanbul = pd.concat([df_istanbul, new_row], ignore_index=True)
    driver.quit()

In [13]:
df_istanbul['Sehir']='Istanbul'

In [14]:
df_istanbul.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,[3.700.000 TL],19 Aralık 2024,95,100m²,2+1,1,5,2,11-15 arası,İkinci El,...,Evet,Kat Mülkiyeti,None,Eşyasız,Kat Mülkiyeti,Danışman,None,None,Ara Kat,Istanbul
1,[6.500.000 TL],18 Aralık 2024,95,120m²,3+1,2,9,2,31-35 arası,None,...,Evet,None,None,Eşyasız,None,None,None,None,Ara Kat,Istanbul
2,[5.500.000 TL],17 Aralık 2024,90,119m²,2+1,2,10-20 arası,10,0,Sıfır,...,Evet,None,None,None,None,None,None,None,None,Istanbul
3,[2.700.000 TL],17 Aralık 2024,120,130m²,3+1,1,5,1,31-35 arası,İkinci El,...,Evet,Kat Mülkiyeti,None,Eşyasız,Kat Mülkiyeti,Danışman,None,None,Ara Kat,Istanbul
4,[25.000.000 TL],16 Aralık 2024,160,189m²,3+1,2,4,Yüksek Giriş,21-25 arası,None,...,Hayır,Hisseli Tapu,None,Eşyasız,Hisseli Tapu,None,None,None,None,Istanbul


In [15]:
df_istanbul.to_csv('istanbul.csv', index=False)

## Izmir Sale Homes -Web Scraping

In [21]:
links=[]
for page in range(1,31):
    url = "https://www.zingat.com/izmir-satilik" +'?page='+str(page)
    driver = webdriver.Chrome()  
    driver.get(url) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    soup.prettify()  
    links_html = soup.find_all('a', class_='zl-card-inner')
    for link in links_html:
        new_link ='https://www.zingat.com/'+ link.get('href')
        links.append(new_link)
    driver.quit()

In [22]:
len(links)

630

In [25]:
expected_features = [
    "Fiyat","İlan Tarihi", "Net m²", "Brüt m²", "Oda Sayısı", "Banyo Sayısı",
    "Binadaki Kat Sayısı", "Bulunduğu Kat", "Bina Yaşı", "Yapının Durumu",
    "Isıtma Tipi", "Kira Getirisi", "Aidat", "Kullanım Durumu", "Krediye Uygun","Tapu Durumu","Görüntülü Ev Gezintisi","Mobilya Durumu",
    "Tapu Durumu","Kimden",'Parsel','Ada',"Konut Şekli"
]
df_izmir = pd.DataFrame(columns=expected_features)

In [27]:
for link in links:
    driver = webdriver.Chrome()  
    driver.get(link) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    feature_data = {feature: None for feature in expected_features}
    price_tag = soup.find("strong", {"itemprop": "price"})
    feature_data['Fiyat']=price_tag
    for li in soup.find_all("li", class_="col-md-6"):
        key_element = li.find("strong", class_="col-md-6")
        value_element = li.find("span", class_="col-md-6")
        if key_element and value_element:
            key = key_element.get_text(strip=True)
            value = value_element.get_text(strip=True)
            if key in feature_data:  
                feature_data[key] = value       
    new_row = pd.DataFrame([feature_data], columns=expected_features)  
    df_izmir = pd.concat([df_izmir, new_row], ignore_index=True)
    driver.quit()

In [28]:
df_izmir['Sehir']='Izmir'

In [33]:
df_izmir.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,[7.250.000 TL],17 Aralık 2024,175,190m²,4+1,2,6,2,None,None,...,Hayır,None,None,None,None,None,None,None,None,Izmir
1,[3.500.000 TL],15 Aralık 2024,90,95m²,2+1,1,5,2,6-10 arası,None,...,Evet,None,None,None,None,Danışman,None,None,None,Izmir
2,[10.750.000 TL],13 Aralık 2024,146,180m²,4+1,2,5,5,31-35 arası,İkinci El,...,Evet,None,None,None,None,None,8,9382,Tek Kat,Izmir
3,[2.150.000 TL],24 Kasım 2024,75,80m²,2+1,1,3,3,5,None,...,Yes,None,None,None,None,None,None,None,None,Izmir
4,[6.500.000 TL],19 Kasım 2024,90,1823m²,2+1,1,1,None,2,İkinci El,...,Evet,None,None,None,None,Danışman,None,None,None,Izmir


In [29]:
df_izmir.to_csv('izmir.csv', index=False)

## Ankara Sale Homes -Web Scraping

In [42]:
links=[]
for page in range(1,31):
    url = "https://www.zingat.com/ankara-satilik" +'?page='+str(page)
    driver = webdriver.Chrome()  
    driver.get(url) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    soup.prettify()  
    links_html = soup.find_all('a', class_='zl-card-inner')
    for link in links_html:
        new_link ='https://www.zingat.com/'+ link.get('href')
        links.append(new_link)
    driver.quit()

In [43]:
len(links)

630

In [46]:
expected_features = [
    "Fiyat","İlan Tarihi", "Net m²", "Brüt m²", "Oda Sayısı", "Banyo Sayısı",
    "Binadaki Kat Sayısı", "Bulunduğu Kat", "Bina Yaşı", "Yapının Durumu",
    "Isıtma Tipi", "Kira Getirisi", "Aidat", "Kullanım Durumu", "Krediye Uygun","Tapu Durumu","Görüntülü Ev Gezintisi","Mobilya Durumu",
    "Tapu Durumu","Kimden",'Parsel','Ada',"Konut Şekli"
]
df_ankara = pd.DataFrame(columns=expected_features)

In [48]:
for link in links:
    driver = webdriver.Chrome()  
    driver.get(link) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    feature_data = {feature: None for feature in expected_features}
    price_tag = soup.find("strong", {"itemprop": "price"})
    feature_data['Fiyat']=price_tag
    for li in soup.find_all("li", class_="col-md-6"):
        key_element = li.find("strong", class_="col-md-6")
        value_element = li.find("span", class_="col-md-6")
        if key_element and value_element:
            key = key_element.get_text(strip=True)
            value = value_element.get_text(strip=True)
            if key in feature_data:  
                feature_data[key] = value       
    new_row = pd.DataFrame([feature_data], columns=expected_features)  
    df_ankara = pd.concat([df_ankara, new_row], ignore_index=True)
    driver.quit()

In [49]:
df_ankara['Sehir']='Ankara'

In [50]:
df_ankara.to_csv('ankara.csv', index=False)

## Antalya Sale Homes -Web Scraping

In [51]:
links=[]
for page in range(1,31):
    url = "https://www.zingat.com/antalya-satilik" +'?page='+str(page)
    driver = webdriver.Chrome()  
    driver.get(url) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    soup.prettify()  
    links_html = soup.find_all('a', class_='zl-card-inner')
    for link in links_html:
        new_link ='https://www.zingat.com/'+ link.get('href')
        links.append(new_link)
    driver.quit()

In [52]:
len(links)

630

In [53]:
expected_features = [
    "Fiyat","İlan Tarihi", "Net m²", "Brüt m²", "Oda Sayısı", "Banyo Sayısı",
    "Binadaki Kat Sayısı", "Bulunduğu Kat", "Bina Yaşı", "Yapının Durumu",
    "Isıtma Tipi", "Kira Getirisi", "Aidat", "Kullanım Durumu", "Krediye Uygun","Tapu Durumu","Görüntülü Ev Gezintisi","Mobilya Durumu",
    "Tapu Durumu","Kimden",'Parsel','Ada',"Konut Şekli"
]
df_antalya = pd.DataFrame(columns=expected_features)

In [54]:
for link in links:
    driver = webdriver.Chrome()  
    driver.get(link) 
    html_content = driver.page_source  
    soup = BeautifulSoup(html_content, "html.parser")
    feature_data = {feature: None for feature in expected_features}
    price_tag = soup.find("strong", {"itemprop": "price"})
    feature_data['Fiyat']=price_tag
    for li in soup.find_all("li", class_="col-md-6"):
        key_element = li.find("strong", class_="col-md-6")
        value_element = li.find("span", class_="col-md-6")
        if key_element and value_element:
            key = key_element.get_text(strip=True)
            value = value_element.get_text(strip=True)
            if key in feature_data:  
                feature_data[key] = value       
    new_row = pd.DataFrame([feature_data], columns=expected_features)  
    df_antalya = pd.concat([df_antalya, new_row], ignore_index=True)
    driver.quit()

In [55]:
df_antalya['Sehir']='Antalya'

In [56]:
df_antalya.to_csv('antalya.csv', index=False)

## Concating each file

In [5]:
df_istanbul=pd.read_csv("istanbul.csv")
df_istanbul["Fiyat"]=df_istanbul['Fiyat'].str.extract(r'content="(\d+)"')
df_istanbul.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu.1,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,3700000,19 Aralık 2024,95.0,100m²,2+1,1,5,2,11-15 arası,İkinci El,...,Evet,Kat Mülkiyeti,NaN,Eşyasız,Kat Mülkiyeti,Danışman,NaN,NaN,Ara Kat,Istanbul
1,6500000,18 Aralık 2024,95.0,120m²,3+1,2,9,2,31-35 arası,NaN,...,Evet,NaN,NaN,Eşyasız,NaN,NaN,NaN,NaN,Ara Kat,Istanbul
2,5500000,17 Aralık 2024,90.0,119m²,2+1,2,10-20 arası,10,0,Sıfır,...,Evet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Istanbul
3,2700000,17 Aralık 2024,120.0,130m²,3+1,1,5,1,31-35 arası,İkinci El,...,Evet,Kat Mülkiyeti,NaN,Eşyasız,Kat Mülkiyeti,Danışman,NaN,NaN,Ara Kat,Istanbul
4,25000000,16 Aralık 2024,160.0,189m²,3+1,2,4,Yüksek Giriş,21-25 arası,NaN,...,Hayır,Hisseli Tapu,NaN,Eşyasız,Hisseli Tapu,NaN,NaN,NaN,NaN,Istanbul


In [7]:
df_ankara=pd.read_csv("ankara.csv")
df_ankara["Fiyat"]=df_ankara['Fiyat'].str.extract(r'content="(\d+)"')
df_ankara.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu.1,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,3000000,16 Aralık 2024,130.0,140m²,3+1,2,4,NaN,6-10 arası,İkinci El,...,Evet,NaN,NaN,Eşyasız,NaN,NaN,NaN,NaN,Ara Kat,Ankara
1,7800000,19 Kasım 2024,180.0,220m²,4+1,2,3,2,Between 11-15,Second Hand,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tek Kat,Ankara
2,13500000,19 Kasım 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Evet,NaN,NaN,NaN,NaN,NaN,6.0,1305,NaN,Ankara
3,1600000,19 Kasım 2024,1357.0,1400m²,3+1,1,1,NaN,16-20 arası,İkinci El,...,Evet,Kat Mülkiyeti,NaN,Eşyasız,Kat Mülkiyeti,NaN,NaN,NaN,NaN,Ankara
4,8250000,19 Kasım 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hayır,Müstakil Parsel,NaN,NaN,Müstakil Parsel,NaN,NaN,NaN,NaN,Ankara


In [9]:
df_izmir=pd.read_csv("izmir.csv")
df_izmir["Fiyat"]=df_izmir['Fiyat'].str.extract(r'content="(\d+)"')
df_izmir.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu.1,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,7250000,17 Aralık 2024,175,190m²,4+1,2,6,2,NaN,NaN,...,Hayır,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Izmir
1,3500000,15 Aralık 2024,90,95m²,2+1,1,5,2,6-10 arası,NaN,...,Evet,NaN,NaN,NaN,NaN,Danışman,NaN,NaN,NaN,Izmir
2,10750000,13 Aralık 2024,146,180m²,4+1,2,5,5,31-35 arası,İkinci El,...,Evet,NaN,NaN,NaN,NaN,NaN,8.0,9382,Tek Kat,Izmir
3,2150000,24 Kasım 2024,75,80m²,2+1,1,3,3,5,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Izmir
4,6500000,19 Kasım 2024,90,1823m²,2+1,1,1,NaN,2,İkinci El,...,Evet,NaN,NaN,NaN,NaN,Danışman,NaN,NaN,NaN,Izmir


In [11]:
df_antalya=pd.read_csv("antalya.csv")
df_antalya["Fiyat"]=df_antalya['Fiyat'].str.extract(r'content="(\d+)"')
df_antalya.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu.1,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,8150000,12 Aralık 2024,150.0,160m²,3+1,2,2,2,16-20 arası,NaN,...,Evet,NaN,NaN,Eşyasız,NaN,NaN,NaN,NaN,NaN,Antalya
1,17500000,21 Kasım 2024,180.0,200m²,3+1,2,2,Müstakil,11-15 arası,NaN,...,Evet,NaN,NaN,Eşyasız,NaN,NaN,NaN,NaN,Bahçe Dubleksi,Antalya
2,20500000,3 Aralık 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hayır,Müstakil Parsel,NaN,NaN,Müstakil Parsel,NaN,NaN,NaN,NaN,Antalya
3,17500000,4 Aralık 2024,180.0,200m²,3+1,2,2,NaN,NaN,NaN,...,Evet,Kat Mülkiyeti,NaN,NaN,Kat Mülkiyeti,NaN,NaN,NaN,NaN,Antalya
4,52000000,17 Aralık 2024,380.0,500m²,4+1,4,2,NaN,0,NaN,...,Evet,NaN,NaN,Eşyasız,NaN,NaN,NaN,NaN,NaN,Antalya


In [13]:
df=pd.concat([df_istanbul,df_izmir,df_ankara,df_antalya])

In [15]:
df.head()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu.1,Kimden,Parsel,Ada,Konut Şekli,Sehir
0,3700000,19 Aralık 2024,95.0,100m²,2+1,1,5,2,11-15 arası,İkinci El,...,Evet,Kat Mülkiyeti,NaN,Eşyasız,Kat Mülkiyeti,Danışman,NaN,NaN,Ara Kat,Istanbul
1,6500000,18 Aralık 2024,95.0,120m²,3+1,2,9,2,31-35 arası,NaN,...,Evet,NaN,NaN,Eşyasız,NaN,NaN,NaN,NaN,Ara Kat,Istanbul
2,5500000,17 Aralık 2024,90.0,119m²,2+1,2,10-20 arası,10,0,Sıfır,...,Evet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Istanbul
3,2700000,17 Aralık 2024,120.0,130m²,3+1,1,5,1,31-35 arası,İkinci El,...,Evet,Kat Mülkiyeti,NaN,Eşyasız,Kat Mülkiyeti,Danışman,NaN,NaN,Ara Kat,Istanbul
4,25000000,16 Aralık 2024,160.0,189m²,3+1,2,4,Yüksek Giriş,21-25 arası,NaN,...,Hayır,Hisseli Tapu,NaN,Eşyasız,Hisseli Tapu,NaN,NaN,NaN,NaN,Istanbul


In [17]:
df.tail()

,Fiyat,İlan Tarihi,Net m²,Brüt m²,Oda Sayısı,Banyo Sayısı,Binadaki Kat Sayısı,Bulunduğu Kat,Bina Yaşı,Yapının Durumu,...,Krediye Uygun,Tapu Durumu,Görüntülü Ev Gezintisi,Mobilya Durumu,Tapu Durumu.1,Kimden,Parsel,Ada,Konut Şekli,Sehir
625,2580000,19 Kasım 2024,80.0,90m²,2+1,1,2,Yüksek Giriş,NaN,Sıfır,...,Evet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tek Kat,Antalya
626,5270000,19 Kasım 2024,123.0,133m²,2+1,2,5,En Üst Kat,NaN,Sıfır,...,Evet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Çatı Dubleksi,Antalya
627,9180000,19 Kasım 2024,55.0,60m²,1+1,1,3,2,NaN,Sıfır,...,Evet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tek Kat,Antalya
628,3250000,19 Kasım 2024,94.0,104m²,2+1,1,3,2,NaN,Sıfır,...,Evet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tek Kat,Antalya
629,18000000,19 Kasım 2024,60.0,NaN,NaN,NaN,7,Giriş Katı,36-40 arası,NaN,...,Evet,İskanı Alınmış,NaN,NaN,İskanı Alınmış,Danışman,NaN,NaN,NaN,Antalya


In [19]:
df.shape

(2520, 24)

In [21]:
df.to_csv('houseforsale.csv', index=False)